<a href="https://colab.research.google.com/github/filo1110/BDC/blob/master/ResNet18FineTuning_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler
import glob
import cv2
import numpy as np
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import timeit
from tqdm import tqdm

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

torch.manual_seed(17)

device = torch.device("cuda" )

In [3]:
from google.colab import drive
drive_path = '/content/drive'
drive.mount(drive_path)

Mounted at /content/drive


In [4]:
dataset_path = os.path.join(drive_path, "MyDrive", "FaceRecognition/Dataset/WebfaceShort/")
print(dataset_path)

/content/drive/MyDrive/FaceRecognition/Dataset/WebfaceShort/


Classe di creazione del dataset, a seconda del dataset che vuoi usare devi commentare e decommentare precise righe di codice a seconda dei numeri che ti ho messo

In [5]:
class CustomDataset(Dataset):
    def __init__(self, transform):
        #1
        #self.imgs_path = "/content/drive/My Drive/FaceRecognition/Dataset/lfw3/"

        #2
        #self.imgs_path = "/content/drive/My Drive/FaceRecognition/Dataset/WebfaceCrop/"

        #3
        self.imgs_path = "/content/drive/My Drive/FaceRecognition/Dataset/WebfaceShort/"

        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        new_dict = dict()
        counter = 0
        for class_path in file_list:
            #1
            #classPathIndex = class_path.find('w3')
            #class_name = class_path[classPathIndex +3:]

            #2
            #classPathIndex = class_path.find('rop')
            #class_name = class_path[classPathIndex +4:]

            #3
            classPathIndex = class_path.find('ort')
            class_name = class_path[classPathIndex +4:]
            
            new_dict.update({class_name : counter})
            counter+=1
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        #self.class_map =torch.tensor( new_dict)
        self.class_map =new_dict
        self.img_dim = (224, 224)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor([class_id])
        return img_tensor, class_id
    
    def numberOfClasses(self):
        return len(self.class_map)   
    

Alcune trasformate che puoi applicare per fare data augmentation

In [6]:
transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            #transforms.RandomCrop((32, 32)), 
            #transforms.ColorJitter(brightness=0.5), 
            #transforms.RandomRotation(degrees=45),
            #transforms.RandomHorizontalFlip(p=0.5),  # Flips the image horizontally with probability 0.5
            #transforms.RandomVerticalFlip(p=0.05),  # Flips image vertically with probability 0.05
            #transforms.RandomGrayscale(p=0.2),  # Converts to grayscale with probability 0.2
            transforms.ToTensor(),  # Finally converts PIL image to tensor so we can train w. pytorch
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Note: these values aren't optimal
        ]
    )

In [7]:
dataset = CustomDataset(transform)

print(dataset.numberOfClasses())

79


In [8]:
batch_size = 64
validation_split = .10
shuffle_dataset = True
random_seed= 42
#random_seed= 30

data_loader = DataLoader(dataset, batch_size, shuffle=True)


# Creating data indices for training and validation splits:n8
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
#train_indices, test_indices = train_indices[split:], train_indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
#test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)
#test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
#                                                sampler=test_sampler)

print(len(train_sampler))

print(len(valid_sampler))
#print(len(test_sampler))


print(len(train_loader))

print(len(validation_loader))

#print(len(test_loader))



6675
741
105
12


In [9]:
def GetModel ():

    model_ft = models.resnet18(pretrained=True)

    num_ftrs = model_ft.fc.in_features

    for module, param in zip(model_ft.modules(), model_ft.parameters()):
        if isinstance(module, nn.BatchNorm2d):
            param.requires_grad = False
    
    headModel = nn.Sequential(
        nn.Linear(num_ftrs, 1024),
        nn.ReLU(),
        nn.Dropout(0.25),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, dataset.numberOfClasses(), bias=True)
    )
    model_ft.fc = headModel

    #model.fc = nn.Linear(in_features=num_ftrs, out_features=dataset.numberOfClasses(), bias=True)
    #model_ft.softmax = nn.Softmax(dim=1)
    model_ft = model_ft.cuda()    


    return model_ft

In [10]:
def FreezeLayer (model_ft, iteration)  :
    count= 0
    itera=0

    for layer in model_ft.children():
        count+=1
    print(count)   
    for layer in model_ft.children():
        itera+=1
        if itera >= count-iteration and itera != 10:
            print('setta parametro a true del layer ', itera)
            for parameter in layer.parameters():
                parameter.requires_grad = True

In [11]:
def train (model_ft, optimizer, scheduler, criterion, adam = False):

    steps = 0
    running_loss = 0
    print_every = len(train_loader)
    allAcc= []
    train_losses,test_losses= [],[]
    epoch_acc=0
    epochs=20

    start = timeit.default_timer()

    best_val_acc = -1.0
    notIncreasing = False
    for epoch in range(epochs):
        
        for inputs, labels in train_loader:
            model_ft.train()
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            if len(train_sampler) % 2 == 0:
                labels= labels.squeeze_()
            else :
                labels = labels.squeeze_(1)
            #inputs = inputs.view(inputs.shape[0], -1)
            inputs= inputs.float()
            optimizer.zero_grad()        
            logps = model_ft.forward(inputs)   
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if steps % print_every == 0:
                test_loss = 0
                accuracy = 0
                model_ft.eval()
                with torch.no_grad():
                    for inputs, labels in validation_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        if len(valid_sampler) % 2 == 0:
                            labels= labels.squeeze_()
                        else :
                            labels = labels.squeeze_(1)
                        inputs= inputs.float()
                        logps = model_ft.forward(inputs)
                        batch_loss = criterion(logps, labels)
                        test_loss += batch_loss.item()

                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                train_losses.append(running_loss/len(train_loader))
                test_losses.append(test_loss/len(validation_loader))                    
                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Test loss: {test_loss/len(validation_loader):.3f}.. "
                      f"Test accuracy: {accuracy/len(validation_loader):.3f}")
                if adam == True:
                    scheduler.step(running_loss)
                trainLoss = running_loss
                running_loss = 0
                epoch_acc=accuracy/len(validation_loader)
                allAcc.append(epoch_acc)
                print('Acc',allAcc )



            #if epoch_acc > best_val_acc:
             #   best_val_acc = epoch_acc
              #  model_ft.state_dict()
               # torch.save(model_ft.state_dict(), 'C:/Users/vasce/Desktop/Codice Tesi Magistrale/provaprecedente.pt')
                #print('Modello salvato')
        if adam == False:
            scheduler.step()  
        if epoch >= 5:
            i=epoch-1
            if (allAcc[i]- allAcc[i-1] <= 0.05
                and allAcc[i]- allAcc[i-2] <= 0.05 
                and allAcc[i]- allAcc[i-3] <= 0.05 ):
                    notIncreasing = True
                    break
        if trainLoss/print_every <= 0.03:
            break
        if notIncreasing == True : break


    stop = timeit.default_timer()

    print('Tempo: ', stop - start) 

    print("Fine training")
    
    return epoch_acc

In [ ]:
countLR = -1
lr= 0.001
bad = True

while bad == True:
    count = 8
    lr = lr / 10
    print('\n\n\n\n\n\n\n-------------------------------------------------\nSGD con learning rate ', lr)
    
    while True:
        print('\n\nTraining numero ', count + 1)
        model= GetModel()
        criterion = nn.CrossEntropyLoss()
        #optimizer= optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)
        FreezeLayer(model, count)
        accuracy = train(model, optimizer, scheduler, criterion, True)
        count+=1
        if count == 10: break
        if accuracy > 0.9: break
    if accuracy > 0.9: bad =False
    countLR -=1








-------------------------------------------------
SGD con learning rate  0.0001


Training numero  9


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

10
setta parametro a true del layer  2
setta parametro a true del layer  3
setta parametro a true del layer  4
setta parametro a true del layer  5
setta parametro a true del layer  6
setta parametro a true del layer  7
setta parametro a true del layer  8
setta parametro a true del layer  9


In [ ]:
countLR = -1
lr= 0.0001
bad = True

while bad == True:
    count = 0
    print('\n\n\n\n\n\n\n-------------------------------------------------\nSGD con learning rate ', lr)
    
    while True:
        print('\n\nTraining numero ', count + 1)
        model= GetModel()
        criterion = nn.CrossEntropyLoss()
        #optimizer= optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)
        FreezeLayer(model, count)
        accuracy = train(model, optimizer, scheduler, criterion, True)
        count+=1
        if count == 10: break
        if accuracy > 0.9: break
    if accuracy > 0.9: bad =False
    lr+= 0.0001
    countLR -=1

In [ ]:
allAcc=[0.365,0.366,0.3675,0.3685,0.3695]

for epoch in range(100):
    print(epoch)
    if epoch == 5:
            i=epoch
            if (allAcc[i]- allAcc[i-1] <= 0.02 
                and allAcc[i]- allAcc[i-2] <= 0.02 
                and allAcc[i]- allAcc[i-3] <= 0.02 
                and allAcc[i]- allAcc[i-4] <= 0.02):
                    print(allAcc[i])
                    print(allAcc[i-1])
                    print(allAcc[i-2])
                    print(allAcc[i-3])
                    print(allAcc[i-4])
                    print('error')
                    break